In [215]:
import pandas as pd
import requests as req
import bs4
import re
import numpy as np
import time

In [220]:
url = 'https://www.vivareal.com.br/venda/sp/ubatuba/?pagina={}'

resp=req.get(url)
soup = bs4.BeautifulSoup(resp.text)
number_of_pages = int(np.round(int(soup.find('strong',{'class':'results-summary__count'}).text.replace('.',''))/36))

enderecos = [end.text for end in soup.find_all('span',{'class':'property-card__address'})]
nomes = [nome.text for nome in soup.find_all('span',{'class':'property-card__title'})]
areas = [area.text for area in soup.find_all('span',{'class':'js-property-card-detail-area'})]
banheiros = [int(re.findall(r"\d+",banheiro.text.replace('-',' '))[-1]) if len(re.findall(r"\d+",banheiro.text.replace('-',' '))) != 0 else 0 for banheiro in soup.find_all('li',{'class':'property-card__detail-bathroom'})] 
vagas = [int(re.findall(r"\d+",vaga.text.replace('-',' '))[-1]) if len(re.findall(r"\d+",vaga.text.replace('-',' '))) != 0 else 0 for vaga in soup.find_all('li',{'class':'property-card__detail-garage'})] 
precos_condominio = [re.findall(r'\d+.\d+.\d+|\d+',preco.text) for preco in soup.find_all('section',{'class':'property-card__values'})]
precos = [float(preco[0].replace('.','')) for preco in precos_condominio]
condominio = [float(preco[1]) if len(preco) == 2 else 0 for preco in precos_condominio]

lst = list(zip(enderecos,nomes,areas,quartos,banheiros,vagas,precos,condominio))

df_raw = pd.DataFrame(lst)
df_raw.columns = ['endereco','nome','area','quartos','banheiros','vagas','preco','condominio']

i=2

while i <= number_of_pages:
    print(i)
    urlf = url.format(i)
    resp=req.get(urlf)
    if resp.status_code != 200:
        break
    else:
        enderecos = [end.text for end in soup.find_all('span',{'class':'property-card__address'})]
        nomes = [nome.text for nome in soup.find_all('span',{'class':'property-card__title'})]
        areas = [area.text for area in soup.find_all('span',{'class':'js-property-card-detail-area'})]
        banheiros = [int(re.findall(r"\d+",banheiro.text.replace('-',' '))[-1]) if len(re.findall(r"\d+",banheiro.text.replace('-',' '))) != 0 else 0 for banheiro in soup.find_all('li',{'class':'property-card__detail-bathroom'})] 
        vagas = [int(re.findall(r"\d+",vaga.text.replace('-',' '))[-1]) if len(re.findall(r"\d+",vaga.text.replace('-',' '))) != 0 else 0 for vaga in soup.find_all('li',{'class':'property-card__detail-garage'})] 
        precos_condominio = [re.findall(r'\d+.\d+.\d+|\d+',preco.text) for preco in soup.find_all('section',{'class':'property-card__values'})]
        precos = [float(preco[0].replace('.','')) for preco in precos_condominio]
        condominio = [float(preco[1]) if len(preco) == 2 else 0 for preco in precos_condominio]
    
        lst = list(zip(enderecos,nomes,areas,quartos,banheiros,vagas,precos,condominio))     

        df_aux = pd.DataFrame(lst)
        df_aux.columns = ['endereco','nome','area','quartos','banheiros','vagas','preco','condominio']        
        df_raw = pd.concat([df_raw,df_aux],axis=0)
    i += 1
    time.sleep(1)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230


In [221]:
df_raw.shape

(8280, 8)

In [224]:
df_raw.to_csv('ubatuba_real_imoveis.csv')